# **The Storm Of the Century: Dewpoint**
***

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
import metpy.calc as mpcalc
from scipy import ndimage  
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
import metpy
from metpy.plots import ctables
from matplotlib.colors import LinearSegmentedColormap

## **Date Input**

In [2]:
startYear = 1993
startMonth = 3
startDay = 7
startHour = 0
startMinute = 0
startDateTime = dt(startYear,startMonth,startDay, startHour, startMinute)

endYear = 1993
endMonth = 3
endDay = 15
endHour = 0
endMinute = 0
endDateTime = dt(endYear,endMonth,endDay, endHour, endMinute)

In [3]:
dateList = pd.date_range(startDateTime, endDateTime,freq="6H")
dateList

/tmp/ipykernel_47222/666960345.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dateList = pd.date_range(startDateTime, endDateTime,freq="6H")


DatetimeIndex(['1993-03-07 00:00:00', '1993-03-07 06:00:00',
               '1993-03-07 12:00:00', '1993-03-07 18:00:00',
               '1993-03-08 00:00:00', '1993-03-08 06:00:00',
               '1993-03-08 12:00:00', '1993-03-08 18:00:00',
               '1993-03-09 00:00:00', '1993-03-09 06:00:00',
               '1993-03-09 12:00:00', '1993-03-09 18:00:00',
               '1993-03-10 00:00:00', '1993-03-10 06:00:00',
               '1993-03-10 12:00:00', '1993-03-10 18:00:00',
               '1993-03-11 00:00:00', '1993-03-11 06:00:00',
               '1993-03-11 12:00:00', '1993-03-11 18:00:00',
               '1993-03-12 00:00:00', '1993-03-12 06:00:00',
               '1993-03-12 12:00:00', '1993-03-12 18:00:00',
               '1993-03-13 00:00:00', '1993-03-13 06:00:00',
               '1993-03-13 12:00:00', '1993-03-13 18:00:00',
               '1993-03-14 00:00:00', '1993-03-14 06:00:00',
               '1993-03-14 12:00:00', '1993-03-14 18:00:00',
               '1993-03-

## **ERA5**

In [4]:
reanalysis = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2', 
    consolidated=True
)

In [5]:
reanalysis

<xarray.Dataset> Size: 534TB
Dimensions:                                           (time: 552264,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 37)
Coordinates:
  * latitude                                          (latitude) float32 3kB ...
  * level                                             (level) int64 296B 1 .....
  * longitude                                         (longitude) float32 6kB ...
  * time                                              (time) datetime64[ns] 4MB ...
Data variables: (12/31)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    geopotential                                      (time, level, latitude, longitude) float32 85TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    ...                                                ...
    total_precipitation                               (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    type_of_high_vegetation                           (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    type_of_low_vegetation                            (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    u_component_of_wind                               (time, level, latitude, longitude) float32 85TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind                               (time, level, latitude, longitude) float32 85TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity                                 (time, level, latitude, longitude) float32 85TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

## **Area and Domain**

In [ ]:
lonW = -99 + 360
lonE = -59 + 360
latS = 20
latN = 50
inc=.25
cLat, cLon = (latS + latN)/2, (lonW + lonE)/2
latRange = np.arange(latN,latS-inc,-inc) # expand the data range a bit beyond the plot range
lonRange = np.arange(lonW,lonE+inc,+inc) # Need to match longitude values to those of the coordinate variable
constrainLat, constrainLon = (0.6, 7)
proj_map = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
proj_data = ccrs.PlateCarree() # Our data is lat-lon; thus its native projection is Plate Carree.
res = '50m'

## **Variables**

In [ ]:
plevel=850
Q = reanalysis['specific_humidity'].sel(time=dateList,level=plevel,latitude=latRange,longitude=lonRange)

In [ ]:
lats = Q.latitude
lons = Q.longitude

## **MetPy Calculations**

In [ ]:
Q = Q.metpy.convert_units('g/kg')

In [ ]:
# Dewpoint
td = mpcalc.dewpoint_from_specific_humidity(plevel*units.hPa,Q)
#td = mpcalc.smooth_gaussian(td,20)

In [ ]:
td

In [ ]:
Td=td.metpy.convert_units('degF')

In [ ]:
Td = Td.compute()

## **Graphing**

In [ ]:
Td.min(),Td.max()

In [ ]:
contour_int=5
minTd, maxTd = (0,65)
TdContours = np.arange(minTd, maxTd, contour_int)

In [ ]:
def segment_cmap(cmap_name, start, end):
    
    cmap = plt.get_cmap(cmap_name)
    
    
    colors = cmap(np.linspace(start, end))
    return LinearSegmentedColormap.from_list(f"{cmap_name}_segment_{start}_{end}", colors)


In [ ]:
#segment_cmap = segment_cmap('afmhot_r', 0.2, 0.9)

In [ ]:
cmap='summer_r'
bounds = np.arange(minTd, maxTd, 10)
ticks = bounds

In [ ]:
Td=Td.fillna(0)

In [ ]:
for time in dateList:
    print("Processing", time)
    
    # Create the time strings, for the figure title as well as for the file name.
    timeStr = dt.strftime(time,format="%Y-%m-%d %H%M UTC")
    timeStrFile = dt.strftime(time,format="%Y%m%d%H")
    levstr=f'{plevel}'
    
    tl1 = str('ERA5, Valid at: '+ timeStr)
    tl2 = ("Dewpoint Temperature °F at " + levstr + "mb")
    
    title_line = (tl1 + '\n' + tl2 + '\n')
    
    fig = plt.figure(figsize=(22,14)) # Increase size to adjust for the constrained lats/lons
    ax = plt.subplot(1,1,1,projection=proj_map)
    ax.set_extent ([lonW+constrainLon,lonE-constrainLon,latS+constrainLat,latN-constrainLat])
    ax.add_feature(cfeature.COASTLINE.with_scale(res))
    ax.add_feature(cfeature.STATES.with_scale(res),edgecolor='brown')
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature (cfeature.RIVERS.with_scale(res))
    ax.add_feature (cfeature.LAND.with_scale(res))
    ax.add_feature (cfeature.COASTLINE.with_scale(res))
    ax.add_feature (cfeature.LAKES.with_scale(res))
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature(cfeature.OCEAN)
    
 
    # Contour lines 
    #Temp= ax.contour(lons, lats, vorticity.sel(time=time), levels=VortContours, colors='black', linewidths=3, transform=proj_data)
    #ax.clabel(Temp, inline=1, fontsize=12, fmt='%.0f')

    contourf=ax.contourf(lons,lats,Td.sel(time=time),levels=TdContours,cmap=cmap,transform=proj_data,extend='both')
    #cmap=plt.get_cmap('summer_r')
    #cmap.set_extremes(under='#cede23', over='#4cad2f')
    cbar=plt.colorbar(contourf, ax=ax, ticks=ticks)
    cbar.set_label('Dewpoint °F', rotation=270, labelpad=15)
    
    # Wind barbs
    #skip = 4
    #ax.barbs(lons[::skip],lats[::skip],UKts.sel(time=time)[::skip,::skip].values, VKts.sel(time=time)[::skip,::skip].values, color='black',zorder=2,transform=proj_data)

    title = plt.title(title_line,fontsize=16)